In [1]:
import matplotlib.pyplot as plt
from collections import defaultdict
import py3Dmol
import pandas as pd
import re
import requests
from IPython.display import display, Markdown

In [2]:
base_url = "https://www.ebi.ac.uk/pdbe/"

api_base = base_url + "api/"

summary_url = api_base + 'pdb/entry/summary/'
secondary_structure_url = api_base + 'pdb/entry/secondary_structure/'
ligand_url = api_base + '/pdb/entry/ligand_monomers/'


def make_summary(data):
    """
    This function creates a summary for a PDB entry
    by getting data for an entry, and extracting
    pieces of information
    
    :param data: Dict
    :return: String
    """
    
    pdb_id = ""
    # Certain calls could return multiple PDB entries,
    # but the GET summary call we use in this exercise
    # will always return only one PDB entry
    for key in data.keys():
        pdb_id = key

    # The data is a list of dictionaries, and for the summary information,
    # it is always the first element of the list
    entry = data[pdb_id][0]
    
    # Getting the title of the entry
    title = entry['title']
    
    # Getting the release date of the entry
    release_date = entry['release_date']
    # Formatting the entry to make it more user-friendly
    formatted_release_date = "%s/%s/%s" % (
        release_date[:4], 
        release_date[4:6], 
        release_date[6:])
    
    # Getting the experimental methods
    # Note that there can be multiple methods, so this is a list that
    # needs to be iterated
    experimental_methods = ""
    for experimental_method in entry["experimental_method"]:
        if experimental_methods:
            experimental_methods += " and "
        experimental_methods += experimental_method
        
        
        
    # Getting the assemblies
    assemblies = ''
    for assembly in entry['assemblies']:
        if assembly:
            # Blank out the form if it is a monomer (because it must be homo form)
            if assembly['name'] == 'monomer':
                form = ""
            else:
                form = assembly['form']
                
            if assembly['preferred']:
                
                assemblies += f"Preferred form with assembly ID {assembly['assembly_id']} is a {form}{assembly['name']}\n"
            else:
                assemblies += f"Non-preferred form with assembly ID {assembly['assembly_id']} is a {assembly['form']}{assembly['name']}\n"

    # Getting the author list
    authors = "".join(entry['entry_authors'])
    
#         'assemblies': [{'assembly_id': '1', 'form': 'homo', 'preferred': True, 'name': 'monomer'}, {'assembly_id': '2', 'form': 'homo', 'preferred': False, 'name': 'dimer'}]}]}
    
    
        
     
    # Creating the summary text using all the extracted 
    # information
    summary = f'Entry is titled "{title}" and was released on {formatted_release_date} \n\nThis entry was determined using {experimental_methods} \nThe authors are {authors} \n\nAssembly information -\n{assemblies}'

    return summary

def make_request(url, mode, pdb_id):
    """
    This function can make GET and POST requests to
    the PDBe API
    
    :param url: String,
    :param mode: String,
    :param pdb_id: String
    :return: JSON or None
    """
    if mode == "get":
        response = requests.get(url=url+pdb_id)
    elif mode == "post":
        response = requests.post(url, data=pdb_id)

    if response.status_code == 200:
        return response.json()
    else:
        print("[No data retrieved - %s] %s" % (response.status_code, response.text))
    
    return None

def get_secondary_structure_ranges(pdb_id=None, pdb_list=None):
    """
    This function calls the PDBe API and retrieves the residue
    ranges of secondary structural elements in a single PDB entry
    or in a list of PDB entries
    
    :param pdb_id: String,
    :param pdb_list: String
    :return: None
    """
    # If neither a single PDB id, nor a list was provided,
    # exit the function
    if not pdb_id and not pdb_list:
        print("Either provide one PDB id, or a list of ids")
        return None
    
    if pdb_id:
        # If a single PDB id was provided, call the API with GET
        data = make_request(secondary_structure_url, "get", pdb_id)
    else:
        # If multiple PDB ids were provided, call the API with POST
        # The POST API call expects PDB ids as a comma-separated lise
        pdb_list_string = ", ".join(pdb_list)
        data = make_request(secondary_structure_url, "post", pdb_list_string)
        
    # When no data is returned by the API, exit the function
    if not data:
        print("No data available")
        return None
    
    # Loop through all the PDB entries in the retrieved data
    for entry_id in data.keys():
        entry = data[entry_id]
        molecules = entry["molecules"]
        
        # Loop through all the molecules of a given PDB entry
        for i in range(len(molecules)):
            chains = molecules[i]["chains"]
            
            # Loop through all the chains of a given molecules
            for j in range(len(chains)):
                secondary_structure = chains[j]["secondary_structure"]
                helices = secondary_structure["helices"]
                strands = secondary_structure["strands"]
                helix_list = []
                strand_list = []
                
                # Loop through all the helices of a given chain
                for k in range(len(helices)):
                    start = helices[k]["start"]["residue_number"]
                    end = helices[k]["end"]["residue_number"]
                    helix_list.append("%s-%s" % (start, end))
                
                # Loop through all the strands of a given chain
                for l in range(len(strands)):
                    start = strands[l]["start"]["residue_number"]
                    end = strands[l]["end"]["residue_number"]
                    strand_list.append("%s-%s" % (start, end))
                    
                report = "%s chain %s has " % (entry_id, chains[j]["chain_id"])
                if len(helix_list) > 0:
                    report += "helices at residue ranges %s " % str(helix_list)
                else:
                    report += "no helices "
                report += "and "
                if len(strand_list) > 0:
                    report += "strands at %s" % str(strand_list)
                else:
                    "no strands"
                print(report)
                
    return None

def get_ligand_information(data):
    pdb_id = ""
    
    summary = ""

    for key in data.keys():
        pdb_id = key
        
    for ligand in data[pdb_id]:
        summary += f"Ligand : {ligand['chem_comp_name']} at position {ligand['author_residue_number']}\n"
        
    return summary
    

def get_entry_from_api(pdb_id, api_url):
    """
    This function will make a call to the PDBe API using
    the PDB id and API url provided as arguments
    
    :param pdb_id: String,
    :param api_url: String
    :return: Dict or None
    """
    if not re.match("[0-9][A-Za-z][A-Za-z0-9]{2}", pdb_id):
        print("Invalid PDB id")
        return None
    
    # Make a GET call to the API URL
    get_request = requests.get(url=api_url+pdb_id)
    
    if get_request.status_code == 200:
        # If there is data returned (with HTML status code 200)
        # then return the data in JSON format
        return get_request.json()
    else:
        # If there is no data, print status code and response
        print(get_request.status_code, get_request.text)
        return None


# As you can hopefully see, the data displayed is very similar to
# what we had in the mock data in previous sections - however,
# this is actual data coming from the PDBe API
    

In [ ]:
print(display(Markdown(f'# Summary - {snakemake.wildcards.dataset}')))

In [ ]:
print (f'Summary sheet - {snakemake.wildcards.dataset}')

In [ ]:
df = pd.read_csv(snakemake.input[0])
non_fragment_df = df[df['Fragment'] == False]
entry_df = df.dropna(axis=1, how='all')

# EC numbers

In [ ]:
ec_nums = list(pd.unique(df['EC_number']))

ec_set = set()

for num in ec_nums:
    for split in str(num).split(";"):
        ec_set.add(split.strip())
    
print (f'The EC numbers found in the current data set are {ec_set}')

# Taxonomic distribution

In [ ]:
taxonomy_cols = ['Taxonomic_lineage_SUPERKINGDOM', 'Taxonomic_lineage_PHYLUM']

for col in taxonomy_cols:
    fig, ax = plt.subplots(figsize=(len(col) / 3 ,10))
    chart = df[col].value_counts().plot.barh(title=col, ax=ax)
    plt.show()

# Annotation distributions

In [ ]:
key_annotation_cols = snakemake.params.annotation_cols
for col in key_annotation_cols:
    if col in entry_df:
        fig, ax = plt.subplots(figsize=(len(col) / 3 ,10))
        chart = entry_df[col].value_counts().plot.barh(title=col, ax=ax)
        plt.show()

In [ ]:
seq_len = df['Length'].describe()
non_fragment_seq_len = non_fragment_df['Length'].describe()


def print_sequence_summarys(seq_summary):
    print (f"Number of sequences : {seq_summary['count']}")
    print (f"Smallest sequnce length : {seq_summary['min']}")
    print (f"Longest sequnce length : {seq_summary['max']}")
    print (f"Average sequnce length : {seq_summary['mean']}")

print ("Sequence statistics for all sequences")
print_sequence_summarys(seq_len)

print ()

if non_fragment_seq_len.all():

    print ("Sequence statistics for sequences (non-fragments)")
    print_sequence_summarys(non_fragment_seq_len)


# Experimental data (BRENDA)

In [ ]:
skip_brenda_cols = [
'BRENDA_CL', 'BRENDA_GI', 
'BRENDA_SN', 'BRENDA_SY', 
'BRENDA_MW', 'BRENDA_SP',
'BRENDA_NSP', 'BRENDA_PM',
'BRENDA_LO', 'BRENDA_SU', 
'BRENDA_PU', 'BRENDA_ST',
'BRENDA_CR', 'BRENDA_CF',
'BRENDA_RN', 'BRENDA_RT',
'BRENDA_ME', 'BRENDA_REFERENCES']


# Only get the columns that start with BRENDA and that we don't want to skip
brenda_cols =[x for x in df if x.startswith("BRENDA") and 'COMMENT' not in x and 'REFS' not in x and 'UNITS' not in x and not any(skip in x for skip in skip_brenda_cols)]

# Drop rows if they don't have at least one entry in one of the BRENDA columns
b_df = df[brenda_cols].dropna(thresh=1)
print (len(brenda_cols))
fig, ax = plt.subplots(figsize=(len(brenda_cols) / 4 ,10))


# Create and save a plot of the BRENDA column counts
if len(b_df) > 0:
    b_df.count().plot.barh(ax=ax)
    display()

# Structural information

In [ ]:
pdb_df = df[['Entry', 'Cross_reference_PDB']].dropna()
pdb_dict = defaultdict(list)
for pdb_entries in pdb_df.itertuples():
    pdb_dict[pdb_entries.Entry] = [x for x in pdb_entries.Cross_reference_PDB.strip().split(";") if len(x) > 0]

total_structures = sum([len(x) for x in pdb_dict.values()])

print (f"There are {total_structures} total structures across {len(pdb_dict.keys())} unique sequences")

print ("Looking at the first few...")

sample_pdb_dict = {k: pdb_dict[k] for k in list(pdb_dict)[:3]}

for prot_id, pdb_ids in sample_pdb_dict.items():
    for pdb_id in pdb_ids:
        
        display(Markdown(f'# {pdb_id} PDB entry information'))

        print(make_summary(get_entry_from_api(pdb_id, summary_url)))
        
        display(Markdown(f'# {pdb_id} Ligand informtion'))    
        print(get_ligand_information(get_entry_from_api(pdb_id, ligand_url)))
        
    
        display(Markdown(f'# {pdb_id} Secondary structure ranges'))
        get_secondary_structure_ranges(pdb_id)
        
        display(Markdown(f'# {pdb_id}'))
        
        view = py3Dmol.view(query=f'pdb:{pdb_id}')
        view.setStyle({'cartoon':{'color':'spectrum'}})
        display(view)


# General data set information

In [ ]:
display(Markdown(f"These are the columns that are available and have at least one entry"))

for x in entry_df.keys():
    if not x.startswith("BRENDA"):
        print (x)